This notebook is purely for making Hadley Cell indices (HC Indices). Because of our focus on the Southern hemisphere it only takes DJF into account. 

Procedure:
    1. Read netcdf containing monthly vwnd from 1948 to 2017
    2. Sort DJF 
    3. Identify cells using DB clustering 
    4. take spatial means over the identified cells for each timestep. 

In [1]:
#################################################################
#############   Import all necessary packages    ################
#################################################################

import importlib
import plotly
import netCDF4 as nc
from sklearn.cluster import DBSCAN
import pandas as pd

import matplotlib.pyplot as plt
import plotly.plotly as py 
import plotly.tools as tls
from plotly.graph_objs import * 

import numpy as np 
from scipy.io import netcdf 
from scipy.stats import pearsonr
from mpl_toolkits.basemap import Basemap

plotly.tools.set_credentials_file(username='ncresswell', api_key='XVFWb00wZKWyDJTrB2Dl')

%load_ext autoreload
%autoreload 2

In [2]:
###################################################################
################   define reference variables    ##################
###################################################################

#reference points for defining various cells 
s_atlantic_ref = [-22.5,14.4]
s_pacific_ref  = [-17.5,-75.5]
s_indian_ref   = [-26.14, 113.15]
n_atlantic_ref = [21.5, -16]
n_pacific_ref  = [38, -123]

#specify useful variables before running 
filename = 'C:\\Users\\Nathaniel\\Desktop\\Summer2018\\HC_Summer2018\\Data\\vwnd.mon.mean.nc'
alt = 1000 
hadley_threshold = 4

#define clustering data
db_epsilon     = 4.7
db_min_samples = 4

In [20]:
###########################################################################
### create useful functions and enter relevent parameters for indexing  ###
###########################################################################

#see_file_data will receive a file with its path as a string and print information about the file...thie file must be net cdf 
#              will not return anything 
def see_file_data(path):
    temp = nc.Dataset(filename,'r')
    print('FILENAME: ', filename, '\n','\n')
    print('FILE HEADER: ', '\n', temp,'\n','\n')
    print('FILE VARIABLES: ','\n',temp.variables)

    return

#fix_lon will be given a data array and a longitude array 
#        will return arrays of longitude and data that have been reformatted
def fix_lon(array_to_fix, lon_of_array):
    
    #make longitude from -180 to 180 degrees east 
    tmp_lon = lon_of_array
    for n in range (tmp_lon.size):
        if tmp_lon[n]>180:
            tmp_lon[n] = lon_of_array[n]-360
    
    #reorient data to be centered at 0 degrees east 
    i_west    = np.where(tmp_lon<0)
    i_east    = np.where(tmp_lon>0)
    west      = tmp_lon[i_west]
    east      = tmp_lon[i_east]
    fixed_lon = np.array(np.hstack((west,east)))
    
    #make similar adjustments so that vwnd matches new longitude 
    vwnd_west   = np.squeeze(array_to_fix[:,:,i_west])
    vwnd_east   = np.squeeze(array_to_fix[:,:,i_east])
    fixed_array = np.concatenate((vwnd_west,vwnd_east), axis=2)
        
    return fixed_array,fixed_lon

#find_DJF_JJA will take an array of data and an array of 
#             will return 3D arrays of DJF and JJA arrays ----> Assumes Jan start 
def find_DJF_JJA(array_to_parse):
                
    DJF  = np.array([])
    JJA  = np.array([])
    
    for n in range (0,array_to_parse.shape[0]):
        
        temp = None
        
        if (np.mod(n,12)==1 or np.mod(n,12)==11 or np.mod(n,12)==0):
                      
            if (DJF.size==0):
                DJF  = array_to_parse[n,:,:]
                DJF  = DJF.reshape(1,array_to_parse.shape[1],array_to_parse.shape[2])
            else:
                temp = array_to_parse[n,:,:].reshape(1,array_to_parse.shape[1],array_to_parse.shape[2])
                DJF  = np.concatenate((DJF,temp),0)
   
        if (np.mod(n,12)==5 or np.mod(n,12)==6 or np.mod(n,12)==7):
            if (JJA.size==0):
                JJA  = array_to_parse[n,:,:]
                JJA  = JJA.reshape(1,array_to_parse.shape[1],array_to_parse.shape[2])
            else:
                temp = array_to_parse[n,:,:].reshape(1,array_to_parse.shape[1],array_to_parse.shape[2])
                JJA  = np.concatenate((JJA,temp),0)
        temp = None

    return DJF,JJA

#read_nc_file will be given the path to a netcdf file
#             will return an array of time, level, lat, lon, data
#                  >lon, lat and data will be adjusted so lon is from -180 to 180 degrees east, lat is 0 to 360 degrees north
def read_nc_file( filename ):
    
    print('Reading netCDF file...')
    
    #extract lon, lat, level and data stored in the netCDF file specified 
    with nc.Dataset(filename,'r') as f:
        lon   = f.variables['lon'][::]
        lat   = f.variables['lat'][::-1]
        time  = f.variables['time'][::]
        level = f.variables['level'][::]
        temp0 = f.variables['vwnd'][:,:,::-1,:]

    #find index of appropriate pressure 
    index = np.where(level == alt)
    
    temp1 = temp0[:,index,:,:]
    vwnd  = temp1.squeeze()
      
    vwnd,lon = fix_lon(vwnd,lon)
    
    return time, level, lat, lon, vwnd

#find_clusters_labels will receive matrix of mean wind data
#                     will return points that have been clustered (using density-base clustering) and their respective labels 
def find_clusters_labels(data):
    
    print('finding clusters with epsilon: ', db_epsilon,' for ', db_min_samples, ' minimum samples with threshold of ', hadley_threshold,'...')
    
    mask_coordinates = np.empty([2,1])
    
    first_element = True
    
    for n in range(0,data.shape[0]):
        for m in range(0, data.shape[1]):
            if np.absolute(data[n,m])>hadley_threshold:
                temp = np.array([[lat[n]],[lon[m]]])
                if first_element: 
                    mask_coordinates = temp 
                    first_element = False 
                else: 
                    mask_coordinates = np.append(mask_coordinates,temp,1)
    
    mask_coordinates = np.transpose(mask_coordinates)
    db = DBSCAN(eps=db_epsilon, min_samples=db_min_samples).fit(mask_coordinates)
    
    return mask_coordinates, db.labels_

#get_label will receive a reference point and an array of cluster points and their labels
#          will return an integer representing which cluster is associated with the given reference point 
def get_label(ref_point, cluster_points, cluster_point_labels):
    
    print('finding cluster label of cell in question...')
    distances = np.empty([cluster_points.shape[0],1])
    
    for n in range(0, cluster_points.shape[0]):
        temp         = np.linalg.norm(cluster_points[n,:]-ref_point)
        distances[n] = temp
    
    dist_min  = np.amin(distances)
    min_index = np.where(distances == dist_min)
    
    return cluster_point_labels[min_index[0]]

#center_zero takes an array and returns on with the mean removed 
def center_zero(array):
    
    mean = np.mean(array)
    return np.subtract(array,mean)

#get_mask_trace will return an array like data that is populated with 1's when a point is in the 
#      same cluster as the ref point and 0's at all other places
def get_mask_trace(DJF_mean, JJA_mean, lat ,lon , ref_point):
        
    if ref_point[0]>0:
        data = JJA_mean
    elif ref_point[0]<0:
        data = DJF_mean
    else: 
        print('      something went wrong....')
        return
    
    #find cluster points and their labels 
    cluster_coordinates, cluster_labels = find_clusters_labels(data)

    label=get_label(ref_point, cluster_coordinates, cluster_labels)
    mask = np.empty_like(data)
    
    in_cluster = False 
    index = None
    
    for n in range(0,data.shape[0]):
        for m in range(0,data.shape[1]):
            
            for i in range(0,cluster_coordinates.shape[0]):
                
                temp_cluster = cluster_coordinates[i,:]
                temp_point   = np.array([lat[n],lon[m]])
                
                if np.array_equal(temp_cluster, temp_point):
                    in_cluster = True
                    index = i
            
            if in_cluster:
                if cluster_labels[index] == label:
                    mask[n,m] = 1
                else:
                    mask[n,m] = 0
            else: 
                mask[n,m] = 0
            
            in_cluster = False
            index = None
                        
    return mask

#calculate_index will be given data array of JJA or DJF as appropriate and a mask of desired cell
#                will return a time series of indices for the masked cell 
def calculate_index(data, mask):
                 
    #initialize array for later...
    index       = np.empty([1,data.shape[0]])
    
    for m in range(0, data.shape[0]):
        temp                      = data[m,:,:]
        temp                      = np.multiply(data[m,:,:],mask)
        temp[np.where(temp == 0)] = np.nan #turns 0's into nan's to make use of numpy's nanmean function
        
        temp1 = np.nanmean(temp,1)
#         print('***shape of index: ',index.shape)
#         print('***size of index: ',index.size)
#         print('***shape of nan mean temp1 along axis 0: ',np.nanmean(temp1,0).shape)
#         print('***size of nan mean temp1 along axis 1 ',np.nanmean(temp1,0).size)
        index[0,m] = np.nanmean(temp1,0)
    return np.transpose(center_zero(index))

#combine_DJF will take an array during DJF and combine them to represent a single season 
#            will return an array with the same spatial dimensions and 1/3 the time dimension 
def combine_DJF(data):
    
    #initialize new array
    seasonal_data = np.array([np.nan])
    
    #assuming first entry is Jan 
    seasonal_data[0] = np.mean([data[0],data[1]])
    print(seasonal_data[0],'is mean of ', data[0],' and ',data[1])
    #combine the rest of the months 
    for i in range(1,int(np.ceil(np.divide(data.shape[0],3)))):
        
        seasonal_data = np.append(seasonal_data,np.mean([data[(i*3)-1],data[i],data[i+1]]))
        print('seasonal data index: ',i,' is ',seasonal_data[i])
    #return the combined array 
    return seasonal_data

In [21]:
#########################################################################
####  THIS script is used to find indexes for difference hadley cells  ##
#########################################################################

#extract important variables from netCDF file at appropriate level
time, level, lat, lon, vwnd = read_nc_file(filename)

#find arrays for DJF and JJA 
DJF, JJA = find_DJF_JJA(vwnd)
DJF_mean = np.mean(DJF,0)
JJA_mean = np.mean(JJA,0)

# #find indices for south atlantic hadley cell 
print('---->Finding indices for South Atlantic...')
s_atlantic_HC = calculate_index(DJF, get_mask_trace(DJF_mean, JJA_mean, lat ,lon , s_atlantic_ref))

# #find indices for south pacific hadley cell 
print('---->Finding indices for South Pacific...')
s_pacific_HC = calculate_index(DJF, get_mask_trace(DJF_mean, JJA_mean, lat ,lon , s_pacific_ref))

# #find indices for indianocean hadley cell 
print('---->Finding indices for Indian Ocean...')
indian_HC = calculate_index(DJF, get_mask_trace(DJF_mean, JJA_mean, lat ,lon , s_indian_ref))




Reading netCDF file...
---->Finding indices for South Atlantic...
finding clusters with epsilon:  4.7  for  4  minimum samples with threshold of  4 ...
finding cluster label of cell in question...


C:\Users\Nathaniel\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: RuntimeWarning:

Mean of empty slice



[[ 0.5         0.5099945   0.5200043  ...  0.45999146  0.47000122
   0.4900055 ]
 [ 0.16000366  0.17999268  0.19000244 ...  0.11000061  0.11999512
   0.13999939]
 [-0.27000427 -0.19999695 -0.13000488 ... -0.44999695 -0.40000916
  -0.33000183]
 ...
 [ 1.3199921   1.2400055   1.1600037  ...  1.4900055   1.4400024
   1.3899994 ]
 [ 0.5299988   0.47000122  0.41000366 ...  0.69000244  0.6399994
   0.58999634]
 [ 0.1499939   0.05999756 -0.04000854 ...  0.44000244  0.34999084
   0.25      ]]
---->Finding indices for South Pacific...
finding clusters with epsilon:  4.7  for  4  minimum samples with threshold of  4 ...
finding cluster label of cell in question...


C:\Users\Nathaniel\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: RuntimeWarning:

Mean of empty slice



[[ 0.5         0.5099945   0.5200043  ...  0.45999146  0.47000122
   0.4900055 ]
 [ 0.16000366  0.17999268  0.19000244 ...  0.11000061  0.11999512
   0.13999939]
 [-0.27000427 -0.19999695 -0.13000488 ... -0.44999695 -0.40000916
  -0.33000183]
 ...
 [ 1.3199921   1.2400055   1.1600037  ...  1.4900055   1.4400024
   1.3899994 ]
 [ 0.5299988   0.47000122  0.41000366 ...  0.69000244  0.6399994
   0.58999634]
 [ 0.1499939   0.05999756 -0.04000854 ...  0.44000244  0.34999084
   0.25      ]]
---->Finding indices for Indian Ocean...
finding clusters with epsilon:  4.7  for  4  minimum samples with threshold of  4 ...
finding cluster label of cell in question...
[[ 0.5         0.5099945   0.5200043  ...  0.45999146  0.47000122
   0.4900055 ]
 [ 0.16000366  0.17999268  0.19000244 ...  0.11000061  0.11999512
   0.13999939]
 [-0.27000427 -0.19999695 -0.13000488 ... -0.44999695 -0.40000916
  -0.33000183]
 ...
 [ 1.3199921   1.2400055   1.1600037  ...  1.4900055   1.4400024
   1.3899994 ]
 [ 0.52999

C:\Users\Nathaniel\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: RuntimeWarning:

Mean of empty slice



In [27]:
#These indices first element is mean of Jan,feb 1948 and last represents mean of Dec 2016, Jan, Feb 2017. Only DJF is included for SH indices  
combine_DJF(s_atlantic_HC).dump(r'C:\Users\Nathaniel\Desktop\Summer2018\HC_Summer2018\Data\SavedVariables\s_atlantic_indices_YEARLY.npy')
combine_DJF(s_pacific_HC).dump(r'C:\Users\Nathaniel\Desktop\Summer2018\HC_Summer2018\Data\SavedVariables\s_pacific_indices_YEARLY.npy')
combine_DJF(indian_HC).dump(r'C:\Users\Nathaniel\Desktop\Summer2018\HC_Summer2018\Data\SavedVariables\s_indian_indices_YEARLY.npy')

-0.03724548691197427 is mean of  [0.20843318]  and  [-0.28292415]
seasonal data index:  1  is  0.3526152225962858
seasonal data index:  2  is  0.44093245790715824
seasonal data index:  3  is  0.019523963593600275
seasonal data index:  4  is  0.09697280850326824
seasonal data index:  5  is  0.7581900211802702
seasonal data index:  6  is  0.23542883521632163
seasonal data index:  7  is  0.47647621757105796
seasonal data index:  8  is  0.7876701605947396
seasonal data index:  9  is  0.7023604961863738
seasonal data index:  10  is  0.6779767923187793
seasonal data index:  11  is  0.8326198510956347
seasonal data index:  12  is  0.7073970091970345
seasonal data index:  13  is  0.7452626479299447
seasonal data index:  14  is  0.8675397488108855
seasonal data index:  15  is  0.6153574240835091
seasonal data index:  16  is  0.22612367596542646
seasonal data index:  17  is  0.2647064777842741
seasonal data index:  18  is  0.21197591748153974
seasonal data index:  19  is  0.031230156881767595
se

In [26]:
s_atlantic_HC.shape

(209, 1)

In [24]:
temp[0]

-0.03724548691197427